In [4]:
import openai
import os
import pandas as pd
import time
from tqdm import tqdm
import pickle

In [5]:
# OpenAI API key goes here
# %env OPENAI_KEY= ""
# openai.api_key = os.getenv("OPENAI_KEY")

openai.api_key = input()

In [11]:
def askMarv(prompt):
    response = openai.Completion.create(
        model="text-davinci-002",
        prompt="You: "+prompt+"\nMarv:", # Need to add token You: for start of tweet and Marv: for start of response
        temperature=0.5,
        max_tokens=60,
        top_p=0.3,
        frequency_penalty=0.5,
        presence_penalty=0.0
    )
    return response


In [4]:
response = askMarv("Thank you so much for a great cross country flight today and what’s even\
    better was the inflight team onboard. Thankful for what you guys do for me as a Concierge \
    Key and can’t wait to finish up the year strong with y’all. Happy Thanksgiving!")

In [5]:
response['choices'][0]['text']

"\n\nYou're welcome! We're glad you had a great flight and we appreciate your kind words. We're looking forward to finishing the year strong with you as well. Happy Thanksgiving!"

In [12]:
data_dir = "../data/"
df = pd.read_csv(data_dir + 'cleaned_Tweets.csv', index_col=0)
# df.head()

In [7]:
pos_Tweets = df.text[df.airline_sentiment == 1]
pos_Tweets.reset_index(inplace=True, drop=True)
pos_Tweets.head()

0     virginamerica plus you have added commercials...
1     virginamerica yes nearly every time i fly vx ...
2          virginamerica well i did not but now i do d
3     virginamerica it was amazing and arrived an h...
4     virginamerica i lt pretty graphics so much be...
Name: text, dtype: object

In [19]:
pos_neu = df[(df.airline_sentiment == 0) | (df.airline_sentiment == 1)]
pos_neu = pos_neu[["airline_sentiment", "text"]]
pos_neu.reset_index(inplace=True)
pos_neu.head()

,index,airline_sentiment,text
0,0,0,virginamerica what dhepburn said
1,1,1,virginamerica plus you have added commercials...
2,2,0,virginamerica i did not today must mean i nee...
3,6,1,virginamerica yes nearly every time i fly vx ...
4,7,0,virginamerica really missed a prime opportuni...


In [10]:
tweets_responses = [askMarv(tweet) for tweet in tqdm(pos_neu)]

# start = time.time()
# for tweet in tqdm(pos_neu):
#     res = askMarv(tweet)
#     tweets_responses.append(res)
    # end = time.time()
    # print(f"--- {end - start} seconds ---", end='\r')
    # time.sleep(1) # Need to sleep because api is rate limited for free trial of OpenAI
    # start = end

100%|██████████| 5462/5462 [2:16:21<00:00,  1.50s/it]  


In [16]:
with open("../results/tweet_responses.pkl", 'wb') as f:
    pickle.dump(tweets_responses, f, pickle.HIGHEST_PROTOCOL)
    
# with open("../results/tweet_responses.pkl", 'rb') as f:
#     tweets_responses = pickle.load(f)

In [21]:
# with open("../results/first53_marv.tsv", 'w', encoding="utf-16") as f:
#     f.write("tweet,response\n")
#     for i in range(len(tweets_responses)):
#         res = tweets_responses[i]["choices"][0]['text']
#         f.write(pos_Tweets[i] +"\t"+ ''.join(res.split(sep='\n')) +"\n")

In [32]:
with open("../results/pos_tweets_responses_marv.tsv", 'w', encoding="utf-16") as f:
    f.write("index\tairline_sentiment\ttweet\tresponse\n")
    for i in range(len(tweets_responses)):
        res = tweets_responses[i]["choices"][0]['text']
        res = ''.join(res.split(sep='\n'))
        query = pos_neu.iloc[i]
        f.write(f"{query['index']}\t{query.airline_sentiment}\t{query.text}\t{res}\n")
